# Automatic Parallelism
Deep learning frameworks (e.g., MXNet and PyTorch) automatically construct computational graphs at the backend. Using a computational graph, the system is aware of all the dependencies, and can selectively execute multiple non-interdependent tasks in parallel to improve speed.

Typically, a single operator will use all the computational resources on all CPUs or on a single GPU. For example, the dot operator will use all cores (and threads) on all CPUs, even if there are multiple CPU processors on a single machine. The same applies to a single GPU. Hence parallelization is not quite so useful for single-device computers.
With multiple devices things matter more. While parallelization is typically most relevant between multiple GPUs, adding the local CPU will increase performance slightly.

In [1]:
import torch
from d2l import torch as d2l

## Parallel Computation on GPUs

In [2]:
devices = d2l.try_all_gpus()
def run(x):
    return [x.mm(x) for _ in range(50)]
x_gpu1 = torch.rand(size=(4000, 4000), device=devices[0])
# x_gpu2 = torch.rand(size=(4000, 4000), device=devices[1])

IndexError: list index out of range

In [5]:
run(x_gpu1)
# run(x_gpu2)  # Warm-up all devices
torch.cuda.synchronize(devices[0])
# torch.cuda.synchronize(devices[1])

with d2l.Benchmark('GPU1 time'):
    run(x_gpu1)
    torch.cuda.synchronize(devices[0])
#
# with d2l.Benchmark('GPU2 time'):
#     run(x_gpu2)
#     torch.cuda.synchronize(devices[1])

RuntimeError: CUDA out of memory. Tried to allocate 62.00 MiB (GPU 0; 4.00 GiB total capacity; 2.72 GiB already allocated; 0 bytes free; 2.72 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

If we remove the synchronize statement between both tasks the system is free to parallelize computation on both devices automatically.

## Parallel Computation and Communication
In many cases we need to move data between different devices, say between the CPU and GPU, or between different GPUs. For instance, this occurs when we want to perform distributed optimization where we need to aggregate the gradients over multiple accelerator cards. Let’s simulate this by computing on the GPU and then copying the results back to the CPU.

In [6]:
def copy_to_cpu(x, non_blocking=False):
    return [y.to('cpu', non_blocking=non_blocking) for y in x]

with d2l.Benchmark('Run on GPU1'):
    y = run(x_gpu1)
    torch.cuda.synchronize()

with d2l.Benchmark('Copy to CPU'):
    y_cpu = copy_to_cpu(y)
    torch.cuda.synchronize()

Run on GPU1: 1.8090 sec


RuntimeError: CUDA out of memory. Tried to allocate 62.00 MiB (GPU 0; 4.00 GiB total capacity; 2.72 GiB already allocated; 0 bytes free; 2.72 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

# Summary
Modern systems have a variety of devices, such as multiple GPUs and CPUs. They can be used in parallel, asynchronously.
Modern systems also have a variety of resources for communication, such as PCI Express, storage (typically solid-state drives or via networks), and network bandwidth. They can be used in parallel for peak efficiency.
The backend can improve performance through automatic parallel computation and communication.